# Black-Litterman (Bayesian Machinery)
Reproducing the original Black-Litterman (Goldman Sachs working) paper.

In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

In [2]:
def as_colvec(x):
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x, axis=1)

In [3]:
np.arange(4)

array([0, 1, 2, 3])

In [4]:
# test out the function
as_colvec(np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

In [5]:
# implied returns from the market portfolio weights
def implied_returns(delta, sigma, w):
    '''
    Obtain the implied expected returns by reverse engineering the weights
    Inputs:
        delta: Risk Aversion Coefficient (scalar)
        sigma: Variance-Covariance Matrix (N x N) as DataFrame
        w: Portfolio weights (N x 1) as Series
    Returns an N x 1 vector of returns as Series
    '''
    ir = delta * sigma.dot(w).squeeze() # squeeze() turns one column dataframe to series
    ir.name = 'Implied Returns'
    return ir

In [6]:
# simplifying assumption that omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    '''
    Returns the He-Litterman simplified Omega
    Inputs:
        sigma: N x N Covariance Matrix as DataFrame
        tau: a scalar
        p: a K x N DataFrame linking Q and Assets
    Returns a P x P DataFrame, a matrix representing prior uncertainties 
    '''
    # can use .dot() or @ for matrix multiplication
    helit_omega = p.dot(tau * sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)), index=p.index, columns=p.index)

In [7]:
def bl(w_prior, sigma_prior, p, q, omega=None, delta=2.5, tau=0.02):
    '''
    Computes the posterior expected returns based on the original black litterman reference model
    Inputs:
        w_prior must be an N x 1 vector of weights, a series
        sigma_prior is an N x N covariance matrix, a DataFrame
        p must be a K x N matrix linking Q and the Assets, a DataFrame
        q must be K x 1 vector of views, a Series
        omega must be a K x K matrix, a DataFrame or None
        if omega is None, we assume it is proportional to variance of the prior
        delta and tau are scalars
    '''
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # number of assets
    N = w_prior.shape[0]
    # number of views
    K = q.shape[0]
    # reverse engineer weights to get pi
    pi = implied_returns(delta, sigma_prior, w_prior)
    # scale sigma by uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior
    # posterior estimate of mean by master formula
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu_bl
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return mu_bl, sigma_bl

## Simple Example: Absolute Views
Two asset example.
Covariance matrix given:

INTC 46.0 1.06

PFE 1.06 5.33

----------------------------------------------

Intel (INTC) market capitalization = USD 80B

Pfizer (PFE) market capitalization = USD 100B

----------------------------------------------

Thus, market cap portfolio weights:

Wintc = 80/180 = 44%

Wpfe = 100/180 = 56%

In [8]:
tickers = ['INTC', 'PFE']
s = pd.DataFrame([[46.0, 1.06], [1.06, 5.33]], index=tickers, columns=tickers) * 10e-4
pi = implied_returns(delta=2.5, sigma=s, w=pd.Series([.44, .56], index=tickers))
pi

INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

In [9]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    '''
    Invert the dataframe by inverting the underlying matrix
    '''
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)


def w_msr(sigma, mu, scale=True):
    '''
    No constrains on this max-sharpe ratio (unlike the previous one we wrote, which is constrained)
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights by using the Markowitz Optimization Procedure
        mu is the vector of excess expected returns
        sigma must be an N x N matrix as a DataFrame and mu a column vector as a series
    '''
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # assumes all w is positive
    return w

View: The investor expects Intel will return 2% and Pfizer will return 4%.

By estimating the optimal weights obtained by naively implementing Markowitz procedure, the expected returns.

In [10]:
mu_exp = pd.Series([.02, .04], index=tickers)
w_msr(s, mu_exp)

INTC    0.034095
PFE     0.965905
dtype: float64

In [11]:
# implementing black-litterman
# Intel will return 2% and Pfizer will return 4%
q = pd.Series({'INTC': 0.02, 'PFE': 0.04})

# absolute views, the pick matrix
p = pd.DataFrame([
    {'INTC': 1, 'PFE': 0},
    {'INTC': 0, 'PFE': 1}
])

# find expected returns
bl_mu, bl_sigma = bl(w_prior=pd.Series({'INTC': .44, 'PFE': .56}), sigma_prior=s, p=p, q=q)

bl_mu

INTC    0.037622
PFE     0.024111
dtype: float64

In [12]:
# using black letterman returns
w_msr(bl_sigma, bl_mu)

INTC    0.140692
PFE     0.859308
dtype: float64

## Simple Example: Relative Views
Intel will outperform Pfizer by 2%.

In [13]:
# expected returns from cap weights
pi

INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

In [14]:
# implementing black-literman
# Intel will outperform Pfizer by 2%
q = pd.Series([0.02])

# the pick matrix
p = pd.DataFrame([
    {'INTC': 1, 'PFE': -1}
])

# find expected returns
bl_mu, bl_sigma = bl(w_prior=pd.Series({'INTC': .44, 'PFE': .56}), sigma_prior=s, p=p, q=q)

bl_mu

INTC    0.041374
PFE     0.009646
dtype: float64

In [15]:
# expected difference between returns
pi[0]-pi[1]

0.043456

In [16]:
# difference between expected black-litterman returns
bl_mu[0]-bl_mu[1]

0.031728

We can see that the Black-Litterman spread is a blend between the market expected returns and investor view returns.

In [17]:
# using black litterman expected returns and covariance matrix
w_msr(bl_sigma, bl_mu)

INTC    0.347223
PFE     0.652777
dtype: float64

## Reproducing He-Litterman Results
This example involves allocation between 7 countries.

In [18]:
# the countries
countries = ['AU', 'CA', 'FR', 'DE', 'JP', 'UK', 'US']

# the correlation matrix
rho = pd.DataFrame([
    [1.00, 0.488, 0.478, 0.515, 0.439, 0.512, 0.491],
    [0.488, 1.000, 0.664, 0.655, 0.310, 0.608, 0.779],
    [0.478, 0.664, 1.000, 0.861, 0.355, 0.783, 0.668],
    [0.515, 0.655, 0.861, 1.000, 0.354, 0.777, 0.653],
    [0.439, 0.310, 0.355, 0.354, 1.000, 0.405, 0.306],
    [0.512, 0.608, 0.783, 0.777, 0.405, 1.000, 0.652],
    [0.491, 0.779, 0.668, 0.653, 0.306, 0.652, 1.000]
], index=countries, columns=countries)

# the volatilities
vols = pd.DataFrame([0.160, 0.203, 0.248, 0.271, 0.210, 0.200, 0.187], index=countries, columns=['vol'])

# the cap weights
w_eq = pd.DataFrame([0.016, 0.022, 0.052, 0.055, 0.116, 0.124, 0.615], index=countries, columns=['CapWeight'])

# compute covariance matrix
sigma_prior = vols.dot(vols.T) * rho

# compute pi and compare
pi = implied_returns(delta=2.5, sigma=sigma_prior, w=w_eq)
pi

AU    0.039376
CA    0.069152
FR    0.083581
DE    0.090272
JP    0.043028
UK    0.067677
US    0.075600
Name: Implied Returns, dtype: float64

### View 1: Germany vs Rest of Europe
Imposing the view that German equities will outperform the rest of Europe equities by 5%.

In [19]:
# single view
q = pd.Series([.05])

# start with a single view, all zeros and overwrite specific view
p = pd.DataFrame([0.]*len(countries), index=countries).T

# find relative caps of FR and UK to split relative outperformance of DE
w_fr = w_eq.loc['FR']/(w_eq.loc['FR']+w_eq.loc['UK'])
w_uk = w_eq.loc['UK']/(w_eq.loc['FR']+w_eq.loc['UK'])
p.loc[0, 'DE'] = 1.
p.loc[0, 'FR'] = -w_fr.CapWeight
p.loc[0, 'UK'] = -w_uk.CapWeight
p

,AU,CA,FR,DE,JP,UK,US
0,0.0,0.0,-0.295455,1.0,0.0,-0.704545,0.0


In [20]:
delta = 2.5
tau = 0.05
# find black litterman expected returns
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau)
bl_mu

AU    0.043282
CA    0.075758
FR    0.092875
DE    0.110375
JP    0.045062
UK    0.069529
US    0.080694
dtype: float64

In [21]:
# the optimal portfolio
def w_star(delta, sigma, mu):
    return (inverse(sigma).dot(mu))/delta

wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
wstar

AU    0.015238
CA    0.020952
FR   -0.039678
DE    0.354295
JP    0.110476
UK   -0.094617
US    0.585714
dtype: float64

In [22]:
# difference of weights between optimal and equilibrium portfolios (unscaled weights)
w_eq = w_msr(delta*sigma_prior, pi, scale=False)

# display difference
np.round(wstar - w_eq/(1+tau), 3)

AU   -0.000
CA   -0.000
FR   -0.089
DE    0.302
JP   -0.000
UK   -0.213
US    0.000
dtype: float64

### View 2: Canada vs US
Canadian equities will outperform US equities by 3%.

In [23]:
view2 = pd.Series([.03], index=[1])
q = q.append(view2)
pick2 = pd.DataFrame([0.]*len(countries), index=countries, columns=[1]).T
p = p.append(pick2)
p.loc[1, 'CA']=1
p.loc[1, 'US']=-1
p.T

,0,1
AU,0.000000,0.0
CA,0.000000,1.0
FR,-0.295455,0.0
DE,1.000000,0.0
JP,0.000000,0.0
UK,-0.704545,0.0
US,0.000000,-1.0


In [24]:
# computing black-litterman weights
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau)
bl_mu

AU    0.044223
CA    0.087300
FR    0.094796
DE    0.112107
JP    0.046164
UK    0.069718
US    0.074817
dtype: float64

In [25]:
wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
wstar

AU    0.015238
CA    0.418633
FR   -0.034279
DE    0.336020
JP    0.110476
UK   -0.081741
US    0.188034
dtype: float64

In [26]:
# difference of weights between optimal and equilibrium portfolios (unscaled weights)
w_eq = w_msr(delta*sigma_prior, pi, scale=False)

# display difference in weights
np.round(wstar - w_eq/(1+tau), 3)

AU    0.000
CA    0.398
FR   -0.084
DE    0.284
JP    0.000
UK   -0.200
US   -0.398
dtype: float64

### View 3: More bullish Canada vs US
Canadian equities will outperform US equities by 4%.

In [28]:
q[1] = 0.04
q

0    0.05
1    0.04
dtype: float64

P remains unchanged with respect to View 2, only Q was changed.

In [29]:
np.round(p.T)

,0,1
AU,0.0,0.0
CA,0.0,1.0
FR,-0.0,0.0
DE,1.0,0.0
JP,0.0,0.0
UK,-1.0,0.0
US,0.0,-1.0


In [31]:
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau)
bl_mu

AU    0.044493
CA    0.090604
FR    0.095346
DE    0.112603
JP    0.046480
UK    0.069772
US    0.073135
dtype: float64

In [32]:
wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
wstar

AU    0.015238
CA    0.533479
FR   -0.032719
DE    0.330743
JP    0.110476
UK   -0.078023
US    0.073187
dtype: float64

In [33]:
# difference in weights between optimal and equilibrium portfolios (unscaled weights)
w_eq = w_msr(delta*sigma_prior, pi, scale=False)

# display difference in weights
np.round(wstar - w_eq/(1+tau), 3)

AU    0.000
CA    0.513
FR   -0.082
DE    0.278
JP    0.000
UK   -0.196
US   -0.513
dtype: float64

### View 4: Increasing view uncertainty
Demonstrating the effect of omega.

Compute the default value of omega, and then increase the uncertainty associated with the first view alone.

In [34]:
# default Proportional to Prior assumption
omega = proportional_prior(sigma_prior, tau, p)

# double uncertainty associated with View 1
omega.iloc[0, 0] = 2*omega.iloc[0, 0]
p.T

,0,1
AU,0.000000,0.0
CA,0.000000,1.0
FR,-0.295455,0.0
DE,1.000000,0.0
JP,0.000000,0.0
UK,-0.704545,0.0
US,0.000000,-1.0


In [36]:
bl_mu, bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau, omega=omega)
bl_mu

AU    0.043312
CA    0.088738
FR    0.092532
DE    0.106485
JP    0.045873
UK    0.069209
US    0.071501
dtype: float64

In [37]:
wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
wstar

AU    0.015238
CA    0.538906
FR   -0.004814
DE    0.236294
JP    0.110476
UK   -0.011480
US    0.067761
dtype: float64

In [38]:
# difference in weights between optimal and equilibrium portfolios
w_eq = w_msr(delta*sigma_prior, pi, scale=False)

# display difference in weights
np.round(wstar-w_eq/(1+tau), 3)

AU   -0.000
CA    0.518
FR   -0.054
DE    0.184
JP    0.000
UK   -0.130
US   -0.518
dtype: float64